<a href="https://colab.research.google.com/github/nicolycosta/VARMA/blob/main/VARMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Vector Autoregression (VAR) 

### Vector Autoregression (VAR) 

## Vector Autoregression (VAR) 